# Protocol for multi-partite approach
### With a trusted server
In the setting where the clients/Alice can trust the server, which is the most basic setting for the multi-partite case, there is no need for any verification. The protocol works as follows:
1. The server establishes a Bell pair with every client in the network.
2. The server performs the distribution steps, resulting in correction outcomes $m_{s_{i}}$ intended for the clients.
3. For all $i$: client $i$ measures their qubit in the $X$ basis, recording measurement outcome $m_{i}$.
4. For all $i$: server sends correction outcome $m_{s_{i}}$ to client $i$.
5. For all $i$: client $i$ receives correction outcome $m_{s_{i}}$ and flips $m_{i}$ based on the necessary corrections (see notebook for GHZ distribution steps).
6. The clients either give up a portion $p$ of their results to determine $Q_{X} = \frac{1 - \langle X_{\mathcal{N}} \rangle}{2}$, or they take it as pre-determined knowledge (in this case, $p = 0$).
7. Alice encodes her message into the other $(1-p)\times L$ states. She can encode a message that is maximally $(1-h_{2}\left(Q_{X}\right))\times (1-p)\times L$ bits long. This results in her codeword $c$ of length $(1-p)\times L$.
8. Alice encodes her codeword into her measurement outcomes $\{m_{a}\}_{k}$ by flipping every bit according to the relevant bit of the codeword.
9. For all $i$: client $i$ announces its measurement outcomes $\{m_{i}^{k}\}_{k}$.
10. For all $i$: compute every bit of the (faulty) codeword as $c_{k} = \bigoplus_{i} m_{i}^{k}$.

Step 2. and Step 3. can be performed simultaneously. Moreover, step 1,2,3 are repeated $L$ times, while step 4. and 5. can either be performed in unison with step 1,2,3, or can be bunched and send all later.

Disclaimer: the protocol works slightly different, as they use a random subset (of their own choosing) of $m$ rounds to estimate $Q_{X}$.

### With an untrusted server
In the setting where the clients/Alice do not trust the server, they have to perform verification in multiple bases, so that they can obtain an estimate/distance bound to the true GHZ state. For this, they need at least $n$ different measurement settings (associated with $n$ generators of the GHZ stabilizer). However, it is hard for them to orchestrate this (in such a way so that the server/untrusted clients cannot interfere). Therefore, they measure randomly in the $X$ or $Y$ basis; half of all rounds will then be a stabilizer element, and therefore have a fixed parity. Multiple rounds thus gives them enough measurement settings. The protocol is thus as follows:
1. The server establishes a Bell pair with every client in the network.
2. The server performs the distribution steps, resulting in correction outcomes $o_{s_{i}}$ intended for the clients.
3. For all $i$: client $i$ measures their qubit in the $X$- or $Y$-basis, recording measurement outcome $o_{i}$ and basis bit $b_{i}$.
4. For all $i$: server sends correction outcome $o_{s_{i}}$ to client $i$.
5. For all $i$: client $i$ receives correction outcome $o_{s_{i}}$ and flips $o_{i}$ based on the necessary corrections (see notebook for GHZ distribution steps). This correction is dependent both on client number, and $b_{i}$ (i.e. what basis they measured).
6. The clients ask a public source of randomness to announce a subset of $k$ rounds for verification.
7. The clients (in random order) announce their outcomes from the subset and announce that they received the outcomes of everyone else.
8. The clients announce their bases from the subset in random order. They sift the rounds with an even number of $Y$ basis measurements, and compute the verification error rate: the relative number of faulty rounds (i.e. those with the wrong parity, which depends on if there's $4i$ or $4i + 2$ $Y$-basis measurements.)
9. Alice checks if the verification error rate is too high, if so, she doesn't perform the encoding step.
10. If the clients want separate PE rounds, they check another random subset of size $m$ of rounds to estimate the '$Z$-basis' error rate $Q_{Z}$ in a similar fashion. If the clients don't want separate PE, they set $Q_{Z} = Q_{X}$. Note that the labels $Z$ and $X$ are here only to draw to comparison with QKD, but don't indicate the actual measurement bases!
11. Alice encodes her message of maximum length $(\frac{1}{2}) \times (1 - h_{2}\left(Q_{Z}\right) )\times (L - m - k)$ into a error correcting code, resulting in a codeword of length $L - m - k$.
12. Alice encodes her codeword into her measurement outcomes $\{o_{a}\}_{k}$ by flipping every bit according to the relevant bit of the codeword.
13. For all $i$: client $i$ announces its measurement outcomes $\{o_{i}^{k}\}_{k}$ and measurement bases $\{o_{i}^{k}\}_{k}$
14. For all $i$: compute every bit of the (faulty) codeword as $c_{k} = \bigoplus_{i} o_{i}^{k}$. They then decode the ECC and read the message.

Note the factor of $\frac{1}{2}$ in the error correction process. This is because half of the rounds will have an odd number of $Y$ basis measurements, so won't be usable. One could overshoot this fraction (i.e. take something $< \frac{1}{2}$ to be more sure the process will work; a quantative bound can be obtained similar to the statistical correction step of the error rate estimations.)

## Trusted server setting
In the trusted server setting, the length of the announced anonymous message becomes:
$$
(1 - h_{2}\left(Q_{X}\right)) \times (L - m)
$$
Once again, one can perform statistical correction, but by default it is not done.

A run of this function is like so:
```
from programs.GHZ_based_trusted_server.functions import get_number_announced_bits as GHZ_trusted_length
GHZ_trusted_lengths, durations = GHZ_trusted_length(
                                nr_clients = nr_clients,
                                nr_rounds = nr_rounds_list[i],
                                nr_estimation_rounds = nr_PE_rounds_list[i],
                                perform_statcor_PE = do_PE_statist,
                                network_configuration = network_config,
                                nr_runtimes = nr_runtimes,
                                )
```
This function takes as input parameters:
```
nr_clients:             (default 3) Number of clients.
nr_rounds:              (default 1e3) Number of rounds to run.
nr_estimation_rounds:   (default 3e2) Number of rounds to consume to estimate the error rate.
perform_statcor_PE:     (default False) Whether to perform statistical correction to PE ereror rate.
PE_tolerance:           (default 1e-8) Tolerance for statistical correction of PE.
network_configuration:  (default None) Network configuration object from Squidasm.
nr_runtimes:            (default 1) Number of times to run the simulation.
Alice:                  (default None) Client that is Alice.
print_loop_nrs:         (default False) Print the loop number. Get's passed to the server program.
```

It outputs two nested lists: `message_lengths` and `run_times`. Each of these lists will have `nr_runs` items. For the `message_lengths`, every item is the total message length. The `run_times` is the same, but then for the total run times (i.e. the runtime of a single pair extrapolated).

## Untrusted server setting
This is a more complicated setting, because the clients need to measure in different bases, to determine how much the state is indeed like the GHZ state (or rather, how much the announcements are 'random' except for the parity, so that anonymity is guaranteed). The clients cannot arrange this secretly without the server knowing (especially in a setting where Alice might not trust all the clients), so they all measure individually randomly in the $X$ or $Y$ basis. With probability $50$% they will not have an even number of $Y$-basis measurements, so those rounds will not be usable.

After having performed all measurements, the clients ask a public source of randomness to announce a subset of $k$ rounds, which they will use for verification. Unlike QKD, this verification must be annoucned and dealt with before announcing the encoded message: it is used to verify that the outcomes are indeed random (i.e. only fixed overall parity, like the GHZ state), so that Alice is completely safe in altering her measurement outcomes before announcing them (i.e. her announcements remain completely indistinghuisable from all other announcements).

After the selection of the $k$ rounds has been made public, the clients first announce the measurement outcomes. Instead of a simultaneous broadcasting system, they announce one-by-one. This means that the last to announce can alter their outcome based on the other outcomes - essentially they can pick whatever parity they want. It would thus be easy for a malicious client to delay their announcement, artificially bringing down the error rate. To combat this, the clients first have to announce their measurement results, but not yet their measurement bases: it is thus not know at the time of announcement what the parity of the outcomes should be' at this time it is completely random. Once all measurement outcomes are announced, they announce their measurement bases.

From these rounds, it is checked what rounds have an even number of $Y$ measurements. For those rounds, the relative number of incorrect parities is calculated (similar to the trusted setting). Once again, a statistical correction can be performed on these (note that this statistical correction reflects the true number of rounds that are being used for estimation, i.e. roughly half of $k$).

Based on this error rate, $Q_{ver}$, Alice determines if she is okay with going through. If so, the clients use, if they want, another subset of $m$ rounds to estimate the non-verification error rate $Q_{EC}$. As these are essentially the same as the verification rounds, this is not really necessary; they could also take $Q_{EC} = Q_{ver}$. It makes the most sense to not do separate PE for $Q_{EC}$, but do statistical correction only for $Q_{ver}$. The python code allows for all these options separately (i.e. there is a `perform_separate_PE` parameter which is default `False`, and there are separate `perform_statcor_PE` and `perform_statcor_VER` that work independently regardless of the value of `perform_separate_PE`).

Alice can now use a ECC to encode a message into the rest of her outcomes, of which there are $L - m - k$. She needs to use very strong error correction: not only will every bit have an error of $Q_{EC}$, roughly half of the bits will also be completely lost. Hence, she needs two levels of error correction: a bit-flip error channel correction for $Q_{EC}$, and a 50-50 erasure channel correction. The maximum length of the original message is taken to be $(\frac{1}{2}) \times (1 - h_{2}\left(Q_{EC}\right) )\times (L - m - k)$, where the first factor reflects the erasure channel and the second reflects the bit flip error channel.

Alice now encodes her codeword into her outcomes, and everyone announces their outcomes and bases. The rest of the clients can now compute the codeword and decode.

Importantly, there is no step akin to 'privacy amplification' in QKD, to ensure the security levels that are wanted. More research is needed, but I think the following works:
Alice knows that for one round the outcomes have a probability of at most $Q_{ver}$ of not being like the GHZ, i.e. not being random. Assuming independendness (which may or may not be a sound assumption) she can combine the outcomes of two rounds (i.e. summing them modulo two). These now have a probability of $Q_{ver}^{2}$ of not being random. But if everyone else also does this, the parity of all announcements is still fixed, so one can still communicate a bit through this method. $l$ repetition of this obtain a $Q_{ver}^{l}$ probability of not being random. Equating this with $\varepsilon_{anon}$ one gets that Alice needs to combine $l = \frac{\log{\varepsilon_{anon}}}{\log{Q_{ver}}}$ rounds to obtain the desired level of anonymity. 

This results in a announced anonymous message length of:
$$
\frac{1}{\lceil \frac{\log{\varepsilon_{anon}}}{\log{\left( Q_{ver} + \mu_{ver} \right)}} \rceil}\times (\frac{1}{2}) \times (1 - h_{2}\left(Q_{EC} + \mu_{EC}\right) )\times (L - m - k)
$$



A run of this function is like so:
```
from programs.GHZ_based_untrusted_server.functions import get_number_announced_bits as GHZ_untrusted_length
GHZ_untrusted_separate_PE, durations = GHZ_untrusted_length(nr_clients = nr_clients,
                                nr_rounds = nr_rounds_list[i],
                                nr_verification_rounds = nr_VER_rounds_list[i],
                                perform_statcor_VER = do_VER_statist,   
                                VER_tolerance = VER_tolerance,
                                perform_separate_PE = True,
                                nr_estimation_rounds = nr_PE_rounds_list[i],
                                perform_statcor_PE = do_PE_statist,
                                PE_tolerance = PE_tolerance,
                                network_configuration = network_config,
                                nr_runtimes = nr_runtimes,
                                anon_tolerance = epsilon_anon
                                    )
```
This function takes as input parameters:
```
    nr_clients:             (default 3) Number of clients.
    nr_rounds:              (default 1e3) Number of rounds to run.
    nr_verification_rounds: (default 3e2) Number of rounds to consume to perform verification.
    perform_statcor_VER:    (default True) Whether to perform statistical correction to verification error rate.
    VER_tolerance:          (default 1e-8) Tolerance for statistical correction of VER.
    perform_separate_PE:    (default True) Whether to perform PE separately from VER, instead of inferring it from VER.
    nr_estimation_rounds:   (default 3e2) Number of rounds to consume to estimate the error rate.
    perform_statcor_PE:     (default False) Whether to perform statistical correction to PE ereror rate.
    PE_tolerance:           (default 1e-8) Tolerance for statistical correction of PE.
    network_configuration:  (default None) Network configuration object from Squidasm.
    nr_runtimes:            (default 1) Number of times to run the simulation.
    Alice:                  (default None) Client that is Alice.
    print_loop_nrs:         (default False) Print the loop number. Get's passed to the server program.
    anon_tolerance:         (default 1e-8) Level of anonymity; see keyrate calculations.
```

It outputs two nested lists: `message_lengths` and `run_times`. Each of these lists will have `nr_runs` items. For the `message_lengths`, every item is the total message length. The `run_times` is the same, but then for the total run times (i.e. the runtime of a single pair extrapolated).

In [ ]:
## Thoughts for updating/improving the protocol
1. Instead of using a public source of randomness to determine the verification rounds, all clients could pick a random subset themselves, and ask everyone else to announce the outcomes of those rounds. The client itself could then withold their measurement outcome so that no cheating is possible.
2. A malicious client can still cheat if they delay the announcement of their measurement bases: they can artificially make any round that they wish not count, by ensuring that an odd number of $Y$ measurements is 'reached'. The previous approach would remedy this, as every client witholds their own outcomes for their verification rounds.